In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
import copy
import time
from rnn_utils import *

In [2]:
class LSTM_CUSTOM(object):

    def __init__(self, sizes, steps):
        self.num_layers = len(sizes)
        self.sizes = sizes
        
        self.a0 = np.zeros((sizes[1],1))
        self.c0 = np.zeros((sizes[1],1))
        
        self.wf = np.random.randn(sizes[1],sizes[0]+sizes[1])
        self.bf = np.random.randn(sizes[1], 1)
        self.wi = np.random.randn(sizes[1],sizes[0]+sizes[1])
        self.bi = np.random.randn(sizes[1], 1)
        self.wc = np.random.randn(sizes[1],sizes[0]+sizes[1])
        self.bc = np.random.randn(sizes[1], 1)
        self.wo = np.random.randn(sizes[1],sizes[0]+sizes[1])
        self.bo = np.random.randn(sizes[1], 1)
        self.wy = np.random.randn(sizes[2], sizes[1])
        self.by = np.random.randn(sizes[2], 1)     
        
        self.steps = steps
        self.counter = 0

    def feedforward(self, x):
        n_x, m = x[0].reshape(-1,1).shape
        n_y, n_a = self.wy.shape      
        c_next = self.c0
        a_next = self.a0
        for i in range(self.steps):
            concat = np.zeros((n_a + n_x, m))
            concat[: n_a, :] = a_next
            concat[n_a :, :] = x[i].reshape(-1,1)
            ft = sigmoid(np.dot(self.wf, concat) + self.bf)
            it = sigmoid(np.dot(self.wi, concat) + self.bi)
            cct = np.tanh(np.dot(self.wc, concat) + self.bc)
            c_next = ft * c_next + it * cct
            ot = sigmoid(np.dot(self.wo, concat) + self.bo)
            a_next = ot * np.tanh(c_next)
        result = sigmoid(np.dot(self.wy, a_next) + self.by).reshape(-1)
        return result 

    def score(self, X, y):
        total_score=0
        for i in range(X.shape[0]):
            predicted = self.feedforward(X[i])
            actual = y[i]
            total_score += np.sum(np.power(predicted-actual,2))  # mean-squared error
        return total_score/X.shape[0]
    
    def score_vector(self, X, y):
        total_score=0
        for i in range(X.shape[0]):
            predicted = self.feedforward(X[i])
            actual = y[i]
            total_score += np.sum(np.power(predicted-actual,2))
        total_score=total_score/X.shape[0] # mean-squared error
        return (1/(1+total_score))

    def accuracy(self, X, y):
        accuracy = 0
        for i in range(X.shape[0]):
            output = self.feedforward(X[i])
            condition = True
            for j in range(len(output)):
                output[j] = round(output[j])
            for j in range(len(output)):
                if(output[j]!=y[i][j]):
                    condition = False
                    break
            if condition:
                accuracy += 1
        return accuracy / X.shape[0] * 100

In [3]:
class LSTM_ABC:

    def __init__(self, n_pops, net_size, X, y, X_test, y_test, steps):
        self.n_pops = n_pops
        self.steps = steps
        self.net_size = net_size
        self.nets = [LSTM_CUSTOM(self.net_size,self.steps) for i in range(self.n_pops)]
        self.X = X[:]
        self.y = y[:]
        self.X_test = X_test[:]
        self.y_test = y_test[:]
        self.accuracy_train = []
        self.accuracy_test = []
        self.limit = self.n_pops
        self.best = LSTM_CUSTOM(self.net_size,self.steps)
    
    def get_random_point(self, type):
        nn = self.nets[0]
        point_index = 0
        if type == 'wf':
            row = random.randint(0,nn.wf.shape[0]-1)
            col = random.randint(0,nn.wf.shape[1]-1)
            point_index = (row, col)
        elif type == 'bf':
            row = random.randint(0,nn.bf.shape[0]-1)
            col = random.randint(0,nn.bf.shape[1]-1)
            point_index = (row, col)
        elif type == 'wi':
            row = random.randint(0,nn.wi.shape[0]-1)
            col = random.randint(0,nn.wi.shape[1]-1)
            point_index = (row, col)
        elif type == 'bi':
            row = random.randint(0,nn.bi.shape[0]-1)
            col = random.randint(0,nn.bi.shape[1]-1)
            point_index = (row, col)
        elif type == 'wc':
            row = random.randint(0,nn.wc.shape[0]-1)
            col = random.randint(0,nn.wc.shape[1]-1)
            point_index = (row, col)
        elif type == 'bc':
            row = random.randint(0,nn.bc.shape[0]-1)
            col = random.randint(0,nn.bc.shape[1]-1)
            point_index = (row, col)
        elif type == 'wo':
            row = random.randint(0,nn.wo.shape[0]-1)
            col = random.randint(0,nn.wo.shape[1]-1)
            point_index = (row, col)
        elif type == 'bo':
            row = random.randint(0,nn.bo.shape[0]-1)
            col = random.randint(0,nn.bo.shape[1]-1)
            point_index = (row, col)
        elif type == 'wy':
            row = random.randint(0,nn.wy.shape[0]-1)
            col = random.randint(0,nn.wy.shape[1]-1)
            point_index = (row, col)
        elif type == 'by':
            row = random.randint(0,nn.by.shape[0]-1)
            col = random.randint(0,nn.by.shape[1]-1)
            point_index = (row, col)
        return point_index

    def get_all_scores(self,Xc,yc):
        return [net.score(Xc, yc) for net in self.nets]

    def get_all_scores_vector(self,Xc,yc):
        return [net.score_vector(Xc, yc) for net in self.nets]

    def get_all_accuracy(self,Xc,yc):
        return [net.accuracy(Xc, yc) for net in self.nets]
    
    def initialization_phase(self):
        for i in range(self.n_pops):
            wf = copy.deepcopy(self.nets[i].wf)
            for x in range(len(wf)):
                for y in range(len(wf[x])):
                    wf[x][y] = np.min(self.nets[i].wf) + np.random.rand()*(np.max(self.nets[i].wf)-np.min(self.nets[i].wf))
            self.nets[i].wf = copy.deepcopy(wf)
            
            bf = copy.deepcopy(self.nets[i].bf)
            for x in range(len(bf)):
                for y in range(len(bf[x])):
                    bf[x][y] = np.min(self.nets[i].bf) + np.random.rand()*(np.max(self.nets[i].bf)-np.min(self.nets[i].bf))
            self.nets[i].bf = copy.deepcopy(bf)
            
            wi = copy.deepcopy(self.nets[i].wi)
            for x in range(len(wi)):
                for y in range(len(wi[x])):
                    wi[x][y] = np.min(self.nets[i].wi) + np.random.rand()*(np.max(self.nets[i].wi)-np.min(self.nets[i].wi))
            self.nets[i].wi = copy.deepcopy(wi)
            
            bi = copy.deepcopy(self.nets[i].bi)
            for x in range(len(bi)):
                for y in range(len(bi[x])):
                    bi[x][y] = np.min(self.nets[i].bi) + np.random.rand()*(np.max(self.nets[i].bi)-np.min(self.nets[i].bi))
            self.nets[i].bi = copy.deepcopy(bi)
            
            wc = copy.deepcopy(self.nets[i].wc)
            for x in range(len(wc)):
                for y in range(len(wc[x])):
                    wc[x][y] = np.min(self.nets[i].wc) + np.random.rand()*(np.max(self.nets[i].wc)-np.min(self.nets[i].wc))
            self.nets[i].wc = copy.deepcopy(wc)
            
            bc = copy.deepcopy(self.nets[i].bc)
            for x in range(len(bc)):
                for y in range(len(bc[x])):
                    bc[x][y] = np.min(self.nets[i].bc) + np.random.rand()*(np.max(self.nets[i].bc)-np.min(self.nets[i].bc))
            self.nets[i].bc = copy.deepcopy(bc)
            
            wo = copy.deepcopy(self.nets[i].wo)
            for x in range(len(wo)):
                for y in range(len(wo[x])):
                    wo[x][y] = np.min(self.nets[i].wo) + np.random.rand()*(np.max(self.nets[i].wo)-np.min(self.nets[i].wo))
            self.nets[i].wo = copy.deepcopy(wo)
            
            bo = copy.deepcopy(self.nets[i].bo)
            for x in range(len(bo)):
                for y in range(len(bo[x])):
                    bo[x][y] = np.min(self.nets[i].bo) + np.random.rand()*(np.max(self.nets[i].bo)-np.min(self.nets[i].bo))
            self.nets[i].bo = copy.deepcopy(bo)
            
            wy = copy.deepcopy(self.nets[i].wy)
            for x in range(len(wy)):
                for y in range(len(wy[x])):
                    wy[x][y] = np.min(self.nets[i].wy) + np.random.rand()*(np.max(self.nets[i].wy)-np.min(self.nets[i].wy))
            self.nets[i].wy = copy.deepcopy(wy)
            
            by = copy.deepcopy(self.nets[i].by)
            for x in range(len(by)):
                for y in range(len(by[x])):
                    by[x][y] = np.min(self.nets[i].by) + np.random.rand()*(np.max(self.nets[i].by)-np.min(self.nets[i].by))
            self.nets[i].by = copy.deepcopy(by)

    def employeed_phase(self,Xc,yc):
        nets = copy.deepcopy(self.nets)
        for i in range(self.n_pops):
            r = list(range(0,i)) + list(range(i+1,self.n_pops))
            coceg = random.choice(r)
            fi1 = random.uniform(-1,1)
            fi2 = random.uniform(-1,1)
            fi3 = random.uniform(-1,1)
            fi4 = random.uniform(-1,1)
            fi5 = random.uniform(-1,1)
            fi6 = random.uniform(-1,1)
            fi7 = random.uniform(-1,1)
            fi8 = random.uniform(-1,1)
            fi9 = random.uniform(-1,1)
            fi10 = random.uniform(-1,1)
            nn = copy.deepcopy(self.nets[i])
            for t in range(1):
                point1 = self.get_random_point('wf')
                nn.wf[point1] = nn.wf[point1] + fi1*(nn.wf[point1]-self.nets[coceg].wf[point1])
                point2 = self.get_random_point('bf')
                nn.bf[point2] = nn.bf[point2] + fi2*(nn.bf[point2]-self.nets[coceg].bf[point2])
                point3 = self.get_random_point('wi')
                nn.wi[point3] = nn.wi[point3] + fi3*(nn.wi[point3]-self.nets[coceg].wi[point3])
                point4 = self.get_random_point('bi')
                nn.bi[point4] = nn.bi[point4] + fi4*(nn.bi[point4]-self.nets[coceg].bi[point4])
                point5 = self.get_random_point('wc')
                nn.wc[point5] = nn.wc[point5] + fi5*(nn.wc[point5]-self.nets[coceg].wc[point5])
                point6 = self.get_random_point('bc')
                nn.bc[point6] = nn.bc[point6] + fi6*(nn.bc[point6]-self.nets[coceg].bc[point6])
                point7 = self.get_random_point('wo')
                nn.wo[point7] = nn.wo[point7] + fi7*(nn.wo[point7]-self.nets[coceg].wo[point7])
                point8 = self.get_random_point('bo')
                nn.bo[point8] = nn.bo[point8] + fi8*(nn.bo[point8]-self.nets[coceg].bo[point8])
                point9 = self.get_random_point('wy')
                nn.wy[point9] = nn.wy[point9] + fi9*(nn.wy[point9]-self.nets[coceg].wy[point9])
                point10 = self.get_random_point('by')
                nn.by[point10] = nn.by[point10] + fi10*(nn.by[point10]-self.nets[coceg].by[point10])
            if(nn.score_vector(Xc,yc)>nets[i].score_vector(Xc,yc)):
                nn.counter = 0
                nets[i] = copy.deepcopy(nn)
            else:
                nets[i].counter += 1
        self.nets = copy.deepcopy(nets)
        
    def onlooked_phase(self,Xc,yc):
        all_scores_vector = self.get_all_scores_vector(Xc,yc)
        sum = np.sum(all_scores_vector)
        probability = all_scores_vector/sum
        nets = copy.deepcopy(self.nets)
        for i in range(self.n_pops):
            index_solution = np.random.choice(list(range(self.n_pops)),p=probability)
            r = list(range(0,index_solution)) + list(range(index_solution+1,self.n_pops))
            coceg = random.choice(r)
            fi1 = random.uniform(-1,1)
            fi2 = random.uniform(-1,1)
            fi3 = random.uniform(-1,1)
            fi4 = random.uniform(-1,1)
            fi5 = random.uniform(-1,1)
            fi6 = random.uniform(-1,1)
            fi7 = random.uniform(-1,1)
            fi8 = random.uniform(-1,1)
            fi9 = random.uniform(-1,1)
            fi10 = random.uniform(-1,1)
            nn = copy.deepcopy(self.nets[index_solution])
            for t in range(1):
                point1 = self.get_random_point('wf')
                nn.wf[point1] = nn.wf[point1] + fi1*(nn.wf[point1]-self.nets[coceg].wf[point1])
                point2 = self.get_random_point('bf')
                nn.bf[point2] = nn.bf[point2] + fi2*(nn.bf[point2]-self.nets[coceg].bf[point2])
                point3 = self.get_random_point('wi')
                nn.wi[point3] = nn.wi[point3] + fi3*(nn.wi[point3]-self.nets[coceg].wi[point3])
                point4 = self.get_random_point('bi')
                nn.bi[point4] = nn.bi[point4] + fi4*(nn.bi[point4]-self.nets[coceg].bi[point4])
                point5 = self.get_random_point('wc')
                nn.wc[point5] = nn.wc[point5] + fi5*(nn.wc[point5]-self.nets[coceg].wc[point5])
                point6 = self.get_random_point('bc')
                nn.bc[point6] = nn.bc[point6] + fi6*(nn.bc[point6]-self.nets[coceg].bc[point6])
                point7 = self.get_random_point('wo')
                nn.wo[point7] = nn.wo[point7] + fi7*(nn.wo[point7]-self.nets[coceg].wo[point7])
                point8 = self.get_random_point('bo')
                nn.bo[point8] = nn.bo[point8] + fi8*(nn.bo[point8]-self.nets[coceg].bo[point8])
                point9 = self.get_random_point('wy')
                nn.wy[point9] = nn.wy[point9] + fi9*(nn.wy[point9]-self.nets[coceg].wy[point9])
                point10 = self.get_random_point('by')
                nn.by[point10] = nn.by[point10] + fi10*(nn.by[point10]-self.nets[coceg].by[point10])
                
            if(nn.score_vector(Xc,yc)>nets[index_solution].score_vector(Xc,yc)):
                nn.counter = 0
                nets[index_solution] = copy.deepcopy(nn)
            else:
                nets[index_solution].counter += 1
        self.nets = copy.deepcopy(nets)

    def scout_phase(self):
        for i in range(self.n_pops):
            if self.nets[i].counter > self.limit:
                wf = copy.deepcopy(self.nets[i].wf)
                for x in range(len(wf)):
                    for y in range(len(wf[x])):
                        wf[x][y] = np.min(self.nets[i].wf) + np.random.rand()*(np.max(self.nets[i].wf)-np.min(self.nets[i].wf))
                self.nets[i].wf = copy.deepcopy(wf)

                bf = copy.deepcopy(self.nets[i].bf)
                for x in range(len(bf)):
                    for y in range(len(bf[x])):
                        bf[x][y] = np.min(self.nets[i].bf) + np.random.rand()*(np.max(self.nets[i].bf)-np.min(self.nets[i].bf))
                self.nets[i].bf = copy.deepcopy(bf)

                wi = copy.deepcopy(self.nets[i].wi)
                for x in range(len(wi)):
                    for y in range(len(wi[x])):
                        wi[x][y] = np.min(self.nets[i].wi) + np.random.rand()*(np.max(self.nets[i].wi)-np.min(self.nets[i].wi))
                self.nets[i].wi = copy.deepcopy(wi)

                bi = copy.deepcopy(self.nets[i].bi)
                for x in range(len(bi)):
                    for y in range(len(bi[x])):
                        bi[x][y] = np.min(self.nets[i].bi) + np.random.rand()*(np.max(self.nets[i].bi)-np.min(self.nets[i].bi))
                self.nets[i].bi = copy.deepcopy(bi)

                wc = copy.deepcopy(self.nets[i].wc)
                for x in range(len(wc)):
                    for y in range(len(wc[x])):
                        wc[x][y] = np.min(self.nets[i].wc) + np.random.rand()*(np.max(self.nets[i].wc)-np.min(self.nets[i].wc))
                self.nets[i].wc = copy.deepcopy(wc)

                bc = copy.deepcopy(self.nets[i].bc)
                for x in range(len(bc)):
                    for y in range(len(bc[x])):
                        bc[x][y] = np.min(self.nets[i].bc) + np.random.rand()*(np.max(self.nets[i].bc)-np.min(self.nets[i].bc))
                self.nets[i].bc = copy.deepcopy(bc)

                wo = copy.deepcopy(self.nets[i].wo)
                for x in range(len(wo)):
                    for y in range(len(wo[x])):
                        wo[x][y] = np.min(self.nets[i].wo) + np.random.rand()*(np.max(self.nets[i].wo)-np.min(self.nets[i].wo))
                self.nets[i].wo = copy.deepcopy(wo)

                bo = copy.deepcopy(self.nets[i].bo)
                for x in range(len(bo)):
                    for y in range(len(bo[x])):
                        bo[x][y] = np.min(self.nets[i].bo) + np.random.rand()*(np.max(self.nets[i].bo)-np.min(self.nets[i].bo))
                self.nets[i].bo = copy.deepcopy(bo)

                wy = copy.deepcopy(self.nets[i].wy)
                for x in range(len(wy)):
                    for y in range(len(wy[x])):
                        wy[x][y] = np.min(self.nets[i].wy) + np.random.rand()*(np.max(self.nets[i].wy)-np.min(self.nets[i].wy))
                self.nets[i].wy = copy.deepcopy(wy)

                by = copy.deepcopy(self.nets[i].by)
                for x in range(len(by)):
                    for y in range(len(by[x])):
                        by[x][y] = np.min(self.nets[i].by) + np.random.rand()*(np.max(self.nets[i].by)-np.min(self.nets[i].by))
                self.nets[i].by = copy.deepcopy(by)
                
                self.nets[i].counter = 0
                
    def sort_nets(self,Xc,yc):
        # calculate score for each population of neural-net
        score_list = list(zip(self.nets, self.get_all_scores(Xc,yc)))

        # sort the network using its score
        score_list.sort(key=lambda x: x[1])

        # exclude score as it is not needed anymore
        score_list = [obj[0] for obj in score_list]
        self.nets = copy.deepcopy(score_list)
        if(self.best.accuracy(self.X,self.y)<self.nets[0].accuracy(self.X,self.y)):
            self.best = copy.deepcopy(self.nets[0])

    def evolve(self):
        start_time = time.time()
        self.initialization_phase()
        for t in range(25):
            self.accuracy_train.append(self.best.accuracy(self.X,self.y))
            self.accuracy_test.append(self.best.accuracy(self.X_test,self.y_test))
            for i in range(20):
                j1=i*40
                j2=(1+i)*40
                Xc=self.X[j1:j2,:,:]
                yc=self.y[j1:j2,:]
                for k in range(25):  
                    self.employeed_phase(Xc,yc)
                    self.onlooked_phase(Xc,yc)
                    self.scout_phase()
                    self.sort_nets(Xc,yc)
                print("Current iteration : {}, batch : {}".format(t+1,i+1))
                print("Time taken by far : %.1f seconds" % (time.time() - start_time))
                print("Current top member's network score: %.5f " % self.best.score(self.X,self.y))
                print("Current top member's network accuracy: %.2f%%\n" % self.best.accuracy(self.X,self.y))

In [4]:
df = pd.read_csv("../Data/data_(8-8).csv")
sc = StandardScaler()

In [5]:
X = df.iloc[:800, :8].values
y = df.iloc[:800, 8:16].values
X = sc.fit_transform(X)
X_test = df.iloc[800:1000, :8].values
y_test = df.iloc[800:1000, 8:16].values
X_test = sc.fit_transform(X_test)

In [6]:
X1= np.reshape(X, (X.shape[0], X.shape[1], 1))
y1= np.reshape(y, (y.shape[0], y.shape[1]))
X2= np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y2= np.reshape(y_test, (y_test.shape[0], y_test.shape[1]))

In [ ]:
N_POPS = 100
steps = X.shape[1]
NET_SIZE = [1,24,8]
lstm_abc = LSTM_ABC(N_POPS, NET_SIZE, X1, y1, X2, y2, steps)
lstm_abc.evolve()

In [ ]:
lstm_abc.best.accuracy(X1,y1)

In [ ]:
lstm_abc.best.accuracy(X2,y2)

In [ ]:
accuracy_train = np.array(lstm_abc.accuracy_train)
accuracy_test = np.array(lstm_abc.accuracy_test)

In [ ]:
plt.plot(accuracy_train)
plt.plot(accuracy_test )
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()